# Integrantes
- Aguilar Martínez Erick Yair
- Martínez Muñoz Alan Magno
- Mendoza Hernández Carlos Emiliano

Esta implementación requiere de [pydantic](https://docs.pydantic.dev/2.9/) por lo que debe instalarse mediante el siguiente comando:
```sh
pip install pydantic==2.9.0
```

In [2]:
from pydantic import BaseModel, Field
from math import modf
class ExponentOutOfRange(Exception):
    """
    Exception raised when the exponent is out of the IEEE 754 range.
    """
    def __init__(self, message="The exponent is out of the IEEE 754 range"):
        self.message = message
        super().__init__(self.message)
class FloatDTO(BaseModel):
    """
    A class to wrap a floating point number.
    Attributes:
        v (float): The floating point number to be wrapped.
    """
    v : float
class DotPartDTO(BaseModel):
    """
    A class to wrap a decimal fraction.
    Attributes:
        v (float): The decimal fraction to be wrapped.
    """
    v : float = Field(ge=0, lt=1)
class FloatingPointBinaryDTO(BaseModel):
    """
    A class to validate the format of the binary representation.
    Attributes:
        v (str): The binary representation of the floating point number.
    Methods:
        __str__(): Returns a string representation of the object.
        __repr__(): Returns a string representation of the object.
    """
    v: str = Field(pattern="^[01]{32}$")
    def __str__(self):
        return f"s : {self.v[0]} exp : {self.v[1:9]}, mantissa : {self.v[9:]}"
    def __repr__(self) -> str:
        return self.__str__()
class IntegerDTO(BaseModel):
    """
    A class to wrap an integer value within the specified range.

    Attributes:
        v (int): The integer value to be wrapped.

    """
    v : int = Field(ge=-2**31, le=2**31-1)
class BinaryDTO(BaseModel):
    """
    A class to represent a binary number with a minimum length of 1 and only containing 0s and 1s.
    Attributes:
        v (str): The binary number to be
    """
    v : str = Field(pattern="^[01]+$")
class CeroFloatingPointBinaryDTO(FloatingPointBinaryDTO):
    """
    A class to represent a floating point binary number with all bits set to 0.
    """
    def __init__(self):
        super().__init__(v='0'*32)

def zfill(b: BinaryDTO, size : IntegerDTO) -> BinaryDTO:
    """
    Fill a binary string with zeros to the left.
    Parameters:
        b (BinaryDTO): The binary string to be filled.
        size (IntegerDTO): The size of the final binary string. If the size is less than the length of the binary string, the binary string is returned as is.
    Returns:
        BinaryDTO: The binary string filled with zeros to the left.
    Examples:
    >>> zfill(BinaryDTO(v='1'),IntegerDTO(v=8)).v
    '00000001'
    >>> zfill(BinaryDTO(v='101'),IntegerDTO(v=1)).v
    '101'
    """
    return BinaryDTO(v=b.v.zfill(size.v))

def integerToBinary(n: IntegerDTO) -> BinaryDTO:
    """
    Convert an integer to its binary representation.

    Parameters
    ----------
        n (IntegerDTO): The integer to be converted.

    Returns
    -------
        BinaryDTO: The binary representation of the integer.

    Examples:
    >>> integerToBinary(IntegerDTO(v=0)).v
    '0'
    >>> integerToBinary(IntegerDTO(v=63)).v
    '111111'
    >>> integerToBinary(IntegerDTO(v=-5)).v
    '101'
    """
    binStr = ''
    intVal = abs(n.v)
    res = intVal % 2
    binStr += str(res)
    intVal = intVal // 2
    while intVal > 0:
        res = intVal % 2
        binStr += str(res)
        intVal = intVal // 2
    return BinaryDTO(v=binStr[::-1])
def getTheFirstOne(b : BinaryDTO) -> IntegerDTO:
    """
    Get the index of the first occurrence of '1' in the binary string.
    Parameters:
        b (BinaryDTO): The binary string to search for '1'.
    Returns:
        out (IntegerDTO): An object containing the index of the first '1' in the binary string. If no '1' is found, returns -1.
    >>> getTheFirstOne(BinaryDTO(v='1')).v
    0
    >>> getTheFirstOne(BinaryDTO(v='0')).v
    -1
    >>> getTheFirstOne(BinaryDTO(v='00000111110000')).v
    5
    """
    i = 0
    while i < len(b.v) and b.v[i] == '0':
        i+=1
    return IntegerDTO(v=-1 if i == len(b.v) else i)
def dotDecimalToBinary(n: DotPartDTO,maxBits = 23) -> BinaryDTO:
    """
    Convert a decimal fraction to its binary representation.

    Parameters:
        n (DotPartDTO): An object containing the decimal fraction to be converted. The decimal fraction must be between 0 and 1.

    Returns:
    BinaryDTO
        An object containing the binary representation of the decimal fraction.

    Examples:
    >>> dotDecimalToBinary(DotPartDTO(v=0.59375)).v
    '10011000000000000000000'
    >>> dotDecimalToBinary(DotPartDTO(v=0.0)).v
    '00000000000000000000000'
    >>> dotDecimalToBinary(DotPartDTO(v=0.09375)).v
    '00011000000000000000000'
    """
    binStr = ''
    decVal = n.v
    for _ in range(maxBits):
        decVal *= 2
        if decVal >= 1:
            binStr += '1'
            decVal -= 1
        else:
            binStr += '0'
    return BinaryDTO(v=binStr)
def calculateExp(integerPart:BinaryDTO,dotPart:BinaryDTO) -> IntegerDTO:
    """
    Calculate the exponent of a floating point number.

    Parameters:
        integerPart (BinaryDTO): The binary representation of the integer part of the fixed point number.
        dotPart (BinaryDTO): The binary representation of the decimal part of the fixed point number.
    Returns:
        IntegerDTO: The exponent of the floating point number.
    Examples:
    >>> calculateExp(BinaryDTO(v='10011'),BinaryDTO(v='0')).v
    4
    >>> calculateExp(BinaryDTO(v='0000'),BinaryDTO(v='00011')).v
    -4
    >>> calculateExp(BinaryDTO(v='111010'),BinaryDTO(v='1')).v
    5
    """
    i = getTheFirstOne(integerPart)
    if i.v == -1:
        i.v = getTheFirstOne(dotPart).v + 1
        i.v = -i.v
    else:
        i.v = len(integerPart.v) - i.v - 1
    return i
def calculateBiasedExp(exp:IntegerDTO) -> BinaryDTO:
    """
    Calculate the biased exponent of a floating point number with the IEEE 754 standard.

    Parameters:
        exp (IntegerDTO): The exponent of the floating point number.

    Returns:
        BinaryDTO: The biased exponent of the floating point number.
    Examples:
    >>> calculateBiasedExp(IntegerDTO(v=4)).v
    '10000011'
    >>> calculateBiasedExp(IntegerDTO(v=-4)).v
    '01111011'
    >>> calculateBiasedExp(IntegerDTO(v=5)).v
    '10000100'
    """
    isOutOfRange = exp.v < -126 or exp.v > 127
    if isOutOfRange:
        raise ExponentOutOfRange()
    return zfill(integerToBinary(IntegerDTO(v=exp.v+127)),IntegerDTO(v=8))

def calculateMantissa(d:FloatDTO) -> BinaryDTO:
    """
    Calculate the mantissa of a floating point number.

    Parameters:
        d (FloatDTO): The floating point number to calculate the mantissa.

    Returns:
        BinaryDTO: The mantissa of the floating point number.
    Examples:
    >>> calculateMantissa(FloatDTO(v=0.3)).v
    '00110011001100110011001'
    >>> calculateMantissa(FloatDTO(v=0.0087890625)).v
    '00100000000000000000000'
    >>> calculateMantissa(FloatDTO(v=12.25)).v
    '10001000000000000000000'
    """
    MANTISSA_SIZE = 23
    integerPart = BinaryDTO(v = integerToBinary(IntegerDTO(v=int(d.v))).v)
    dotPart = BinaryDTO(v = dotDecimalToBinary(DotPartDTO(v=modf(d.v)[0])).v)
    exp = calculateExp(integerPart,dotPart)
    if integerPart.v == '0':
        newBits = abs(exp.v)
        dotPartExtended = dotDecimalToBinary(DotPartDTO(v=modf(d.v)[0]),MANTISSA_SIZE+newBits)
        return BinaryDTO(v=dotPartExtended.v[newBits:])
    else:
        return BinaryDTO(v=(integerPart.v[1:] + dotPart.v)[:MANTISSA_SIZE])


def dec2float(d: FloatDTO) -> FloatingPointBinaryDTO:
    """
    Convert a floating point number to its binary representation with the IEEE 754 standard.

    Parameters:
        d (FloatDTO): The floating point number to be converted.
    Returns:
        FloatingPointBinaryDTO: The binary representation of the floating point number.
    Examples:
    >>> dec2float(FloatDTO(v=0.00000)).v
    '00000000000000000000000000000000'
    >>> dec2float(FloatDTO(v=0.000001)).v
    '00110101100001100011011110111101'
    >>> dec2float(FloatDTO(v=-0.000001)).v
    '10110101100001100011011110111101'
    """
    if d.v == 0.0:
        return CeroFloatingPointBinaryDTO()
    signBit = '1' if d.v < 0 else '0'
    d.v = abs(d.v)
    integerPartBin = BinaryDTO(v = integerToBinary(IntegerDTO(v=int(d.v))).v)
    dotPartBin = BinaryDTO(v = dotDecimalToBinary(DotPartDTO(v=modf(d.v)[0])).v)
    exp = calculateExp(integerPartBin,dotPartBin)
    biasedExp= calculateBiasedExp(exp)
    mantissa = calculateMantissa(d)
    return FloatingPointBinaryDTO(v = signBit + biasedExp.v + mantissa.v)

def float2dec(f: FloatingPointBinaryDTO) -> FloatDTO:
    """
    Convert a binary representation of a floating point number to its decimal representation.

    Parameters:
        f (FloatingPointBinaryDTO): The binary representation of the floating point number with the IEEE 754 standard.

    Returns:
        FloatDTO: The decimal representation of the floating point number.

    Examples:
    >>> "%.0f" % float2dec(FloatingPointBinaryDTO(v='01010101010101010101010101010101')).v
    '14660154687488'
    >>> "%.8f" % float2dec(FloatingPointBinaryDTO(v='00110101100001100011011110111101')).v
    '0.00000100'
    """
    sign = -1 if f.v[0] == '1' else 1
    exp = int(f.v[1:9],2) - 127
    mantissa = f.v[9:]
    acc = 0.0
    for power,bit in enumerate(mantissa):
        acc += int(bit)*2**(-power-1)
    return FloatDTO(v=sign*(1+acc)*2**exp)


# Testing

In [3]:
import doctest
doctest.testmod(exclude_empty=True,report=True,verbose=False)

TestResults(failed=0, attempted=25)

In [4]:
print(float2dec(dec2float(FloatDTO(v=3.14159265358979323846))))

v=3.141592502593994
